In [ ]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
from datetime import datetime
from tvDatafeed import TvDatafeed, Interval
import multiprocessing
import time
import glob

In [57]:
def get_nasq_syms():
  with open("watchlist_NASDAQ.txt", "r") as f:
    lines = f.readlines()
    f.close()
  line = lines[0].split(",")
  symbols = []
  for l in line:
    x = l.split(":")
    symbols.append(x[1])
  return symbols
def download_data_p(tv,sym):
    data =tv.get_hist(sym,exchange="nasdaq", interval=Interval.in_15_minute, n_bars=50000)
    try:
        data.to_csv("data/{}_1h.csv".format(sym))
    except:
      pass  
def download_data(symbols):
  work_with_dir()
  tv = TvDatafeed()
  for sym in symbols:
    print(symbols.index(sym)+1, "/", len(symbols))
    p = multiprocessing.Process(target=download_data_p, name="dd", args=(tv,sym,))
    p.start()
    time.sleep(2)
    p.terminate()
    p.join()

In [ ]:
download_data(get_nasq_syms())

In [59]:
clmns = [
    "Open 31-30", "Open 30-29", "Open 29-28", "Open 28-27", "Open 27-26", "Open 26-25", "Open 25-24", "Open 24-23", "Open 23-22", "Open 22-21", "Open 21-20", "Open 20-19", "Open 19-18", "Open 18-17", "Open 17-16", "Open 16-15", "Open 15-14", "Open 14-13", "Open 13-12", "Open 12-11", "Open 11-10", "Open 10-9", "Open 9-8", "Open 8-7", "Open 7-6", "Open 6-5", "Open 5-4", "Open 4-3", "Open 3-2", "Open 2-1", 
    "High 31-30", "High 30-29", "High 29-28", "High 28-27", "High 27-26", "High 26-25", "High 25-24", "High 24-23", "High 23-22", "High 22-21", "High 21-20", "High 20-19", "High 19-18", "High 18-17", "High 17-16", "High 16-15", "High 15-14", "High 14-13", "High 13-12", "High 12-11", "High 11-10", "High 10-9", "High 9-8", "High 8-7", "High 7-6", "High 6-5", "High 5-4", "High 4-3", "High 3-2", "High 2-1",
    "Low 31-30", "Low 30-29", "Low 29-28", "Low 28-27", "Low 27-26", "Low 26-25", "Low 25-24", "Low 24-23", "Low 23-22", "Low 22-21", "Low 21-20", "Low 20-19", "Low 19-18", "Low 18-17", "Low 17-16", "Low 16-15", "Low 15-14", "Low 14-13", "Low 13-12", "Low 12-11", "Low 11-10", "Low 10-9", "Low 9-8", "Low 8-7", "Low 7-6", "Low 6-5", "Low 5-4", "Low 4-3", "Low 3-2", "Low 2-1",
    "Close 31-30", "Close 30-29", "Close 29-28", "Close 28-27", "Close 27-26", "Close 26-25", "Close 25-24", "Close 24-23", "Close 23-22", "Close 22-21", "Close 21-20", "Close 20-19", "Close 19-18", "Close 18-17", "Close 17-16", "Close 16-15", "Close 15-14", "Close 14-13", "Close 13-12", "Close 12-11", "Close 11-10", "Close 10-9", "Close 9-8", "Close 8-7", "Close 7-6", "Close 6-5", "Close 5-4", "Close 4-3", "Close 3-2", "Close 2-1",
    "Volume 31-30", "Volume 30-29", "Volume 29-28", "Volume 28-27", "Volume 27-26", "Volume 26-25", "Volume 25-24", "Volume 24-23", "Volume 23-22", "Volume 22-21", "Volume 21-20", "Volume 20-19", "Volume 19-18", "Volume 18-17", "Volume 17-16", "Volume 16-15", "Volume 15-14", "Volume 14-13", "Volume 13-12", "Volume 12-11", "Volume 11-10", "Volume 10-9", "Volume 9-8", "Volume 8-7", "Volume 7-6", "Volume 6-5", "Volume 5-4", "Volume 4-3", "Volume 3-2", "Volume 2-1",
    "suggestion"
]

def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")

  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
def read_txt_list():
  with open("yahoo_stocklist.txt","r")as f:
    lines = f.readlines()
    nlines = []
    for line in lines:
       nlines.append(line.strip())
    return nlines
def read_syms_from_txt():
  with open("syms.txt", "r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols
def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["yes","no"], axis=1)
  y = data[["yes","no"]]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.2)
  print(xTrain.shape, end=" ")
  print(yTrain.shape)
  print(xTest.shape, end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def scaler(row):
    scaler = MinMaxScaler(feature_range=(-6, 6))
    row = scaler.fit_transform(row)
    return row
def process(data):
    data = data.dropna()
    row = []
    if "symbol" in data.columns:
        data.drop("symbol", axis=1, inplace=True)  
    if "Adj Close" in data.columns:
        data.drop("Adj Close", axis=1, inplace=True)
    if "datetime" in data.columns:
        data.drop("datetime",axis=1,inplace=True)
    if len(pd.DataFrame(data).columns) == 7:
      data = data.iloc[:, 1:]
    data = np.array(data)  
    llst = [0, 1, 2, 3, 4]
    grow = []
    srow = []
    for i in range(31, data.shape[0]):
        grow = []
        srow = []

        for j in llst:
           srow.append([
            data[i-1][j] - data[i-2][j],
            data[i-2][j] - data[i-3][j],
            data[i-3][j] - data[i-4][j],
            data[i-4][j] - data[i-5][j],
            data[i-5][j] - data[i-6][j],
            data[i-6][j] - data[i-7][j],
            data[i-7][j] - data[i-8][j],
            data[i-8][j] - data[i-9][j],
            data[i-9][j] - data[i-10][j],
            data[i-10][j] - data[i-11][j],
            data[i-11][j] - data[i-12][j],
            data[i-12][j] - data[i-13][j],
            data[i-13][j] - data[i-14][j],
            data[i-14][j] - data[i-15][j],
            data[i-15][j] - data[i-16][j],
            data[i-16][j] - data[i-17][j],
            data[i-17][j] - data[i-18][j],
            data[i-18][j] - data[i-19][j],
            data[i-19][j] - data[i-20][j],
            data[i-20][j] - data[i-21][j],
            data[i-21][j] - data[i-22][j],
            data[i-22][j] - data[i-23][j],
            data[i-23][j] - data[i-24][j],
            data[i-24][j] - data[i-25][j],
            data[i-25][j] - data[i-26][j],
            data[i-26][j] - data[i-27][j],
            data[i-27][j] - data[i-28][j],
            data[i-28][j] - data[i-29][j],
            data[i-29][j] - data[i-30][j],
            data[i-30][j] - data[i-31][j],
            ])

        for lst in srow:
            mm = np.array(lst)
            mm = np.reshape(mm, (-1, 1))
            grow.append(scaler(mm))

        sugg = "no"
        if data[i][3] > data[i][0]:
            sugg = "yes"

        arr = np.array(grow).flatten()
        arr = np.append(arr, sugg)
        row.append(arr)


    grow = []
    srow = []
    llst = []
    data = []
    arr = []
    mm = []

    return np.array(row)

In [39]:
def each_file_proc(files,now,index):
     data = []
     unattached_dfs = []
     files = list(files)
     for file in files:
        print(f"{files.index(file)+1+index}",end=" ")
        if (files.index(file)+index+1) % 40 == 0:
          print(" ")
        address = f"/content/data/{file}"
        unattached_dfs.append(process(pd.read_csv(address)))
     data = np.array(unattached_dfs[0])
     for i in unattached_dfs[1:]:
           data = np.append(data, np.array(i), axis=0)
        
     unattached_dfs = []
  
     data = pd.DataFrame(data, columns=clmns)
     sugg = data["suggestion"]
     data.drop("suggestion",axis=1,inplace=True)
     sugg = pd.get_dummies(sugg)
     data = pd.concat([data,sugg],axis=1)
     data = data.astype(float)
     right_now = datetime.now().strftime("%H%M%S%f")
     data.to_csv(f"/content/extracted/{now}/{right_now}.csv")  
def extract_data(pieces):
  pd.options.mode.chained_assignment = None
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  files = os.listdir("/content/data/")
  new_files = np.array_split(files, pieces)
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  print("Done")
  index = 0 
  for files in new_files:
     
     each_file_proc(files,now,index)
     index = index + len(files)
  print(" ")
  return now
def delete_all_csv(now):
   path = f'/content/extracted/{now}/*.csv'
   files = glob.glob(path)
   for file in files:
       os.remove(file)
def make_df(now):
   path = f'/content/extracted/{now}/*.parquet'
   files = glob.glob(path)
   #data = pd.DataFrame()
   data = pd.DataFrame()
   for adr in files:
     data =pd.concat([data,pd.read_parquet(adr)])
   if "Unnamed: 0" in data:
     data.drop("Unnamed: 0",axis=1,inplace=True)
   print(data.shape)
   xTrain,xTest,yTrain,yTest = spliting(data)
   data.to_parquet(f'/content/extracted/{now}/data.parquet')
   delete_all_csv(now)
   data = []
   return xTrain,xTest,yTrain,yTest
def to_par(now,howmanyfiles): 
    files = os.listdir(f"/content/extracted/{now}/")
    addresses = []
    for file in files:
      addresses.append(f"/content/extracted/{now}/{file}")
    new_adr = np.array_split(addresses,howmanyfiles)
    for adrs in new_adr:
      datas = []
      for adr in adrs:
        datas.append(pd.read_csv(adr))
      rnow = datetime.now().strftime("%H%M%S%f")
      datas = pd.concat(datas)
      datas.to_parquet(f"/content/extracted/{now}/part_{rnow}.parquet")     

In [ ]:
folder_name = extract_data(10)
to_par(folder_name,1)
xTrain,xTest,yTrain,yTest = make_df(folder_name)

Files In Data : 93
Processing File:
Done
1 2 3 4 5 6 7 8 9 10 11 12 13 

In [ ]:
model = Sequential()



model.add(Dense(1024, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))



opt = tf.keras.optimizers.Adamax()

model.add(Dense(2, activation='sigmoid'))
model.compile(optimizer="adam", loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

In [55]:
model.fit(xTrain,yTrain,epochs=30,batch_size=4096,validation_data=(xTest,yTest),shuffle=False)

Epoch 1/30
84/84 [==============================] - 41s 473ms/step - loss: 0.6016 - accuracy: 0.7051 - val_loss: 0.5933 - val_accuracy: 0.7076
Epoch 2/30
84/84 [==============================] - 39s 469ms/step - loss: 0.5917 - accuracy: 0.7054 - val_loss: 0.5904 - val_accuracy: 0.7076
Epoch 3/30
84/84 [==============================] - 41s 489ms/step - loss: 0.5883 - accuracy: 0.7054 - val_loss: 0.5901 - val_accuracy: 0.7076
Epoch 4/30
84/84 [==============================] - 34s 403ms/step - loss: 0.5857 - accuracy: 0.7056 - val_loss: 0.5906 - val_accuracy: 0.7075
Epoch 5/30
84/84 [==============================] - 40s 482ms/step - loss: 0.5829 - accuracy: 0.7064 - val_loss: 0.5925 - val_accuracy: 0.7057
Epoch 6/30
42/84 [==============>...............] - ETA: 16s - loss: 0.5814 - accuracy: 0.7077

KeyboardInterrupt: ignored